In [1]:
import tensorflow as tf
import numpy as np
import keras
import pickle
from keras import layers
from tensorflow import data
from keras.utils import to_categorical

2025-06-07 17:27:54.465626: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
labellist=['label']
covlist=['subject1','Female','Age','k1a','k2a','k3a','Mono','Gran','CD4T','NK','CD8_naive','CD8pCD28nCD45RAn','PlasmaBlast']
snplist=['subject1','GSA-rs2294942_G','rs1939015_G','rs3762352_C','rs525157_A','rs72660967_T','rs6142884_A']
cpglist=['subject1','cg25929399','cg05704942','cg10306192','cg24088508','cg20089799','cg06961873','cg00704664','cg06961873']
featurelist = covlist+snplist+cpglist
fulllist = featurelist+labellist
# print('covlist',covlist)
csv_batch_size = 100
training_batch_size = 10
# open the serialized schema for the binary data input file
with open("../data/feature_dict.pkl", "rb") as f:
    peg_header_dict = pickle.load(f)
peg_header_dict_subset = {}
for f in fulllist:
    if f in peg_header_dict:
        peg_header_dict_subset[f] = peg_header_dict[f]
        
print('dictionary for tfrecord:',peg_header_dict_subset)


dictionary for tfrecord: {'subject1': FixedLenFeature(shape=[], dtype=tf.string, default_value=''), 'Female': FixedLenFeature(shape=[], dtype=tf.float32, default_value=0.0), 'Age': FixedLenFeature(shape=[], dtype=tf.float32, default_value=0.0), 'k1a': FixedLenFeature(shape=[], dtype=tf.float32, default_value=0.0), 'k2a': FixedLenFeature(shape=[], dtype=tf.float32, default_value=0.0), 'k3a': FixedLenFeature(shape=[], dtype=tf.float32, default_value=0.0), 'Mono': FixedLenFeature(shape=[], dtype=tf.float32, default_value=0.0), 'Gran': FixedLenFeature(shape=[], dtype=tf.float32, default_value=0.0), 'CD4T': FixedLenFeature(shape=[], dtype=tf.float32, default_value=0.0), 'NK': FixedLenFeature(shape=[], dtype=tf.float32, default_value=0.0), 'CD8_naive': FixedLenFeature(shape=[], dtype=tf.float32, default_value=0.0), 'CD8pCD28nCD45RAn': FixedLenFeature(shape=[], dtype=tf.float32, default_value=0.0), 'PlasmaBlast': FixedLenFeature(shape=[], dtype=tf.float32, default_value=0.0), 'GSA-rs2294942_G

In [3]:

# Sample string labels
labels = ['cases','controls']

feature_description = peg_header_dict_subset
def parse_single(example_proto,sublist):
    # Parse the input `tf.train.Example` proto using the dictionary above.
    diction = tf.io.parse_single_example(example_proto, feature_description)
    # convert label to a float
    label = diction['label']
    intlabel = tf.constant(0.0,dtype='float32')
    if label=='cases':
        intlabel = tf.constant(1.0,dtype='float32')
    diction['label'] = intlabel
    filteredlist = {key:diction[key] for key in sublist}
    return filteredlist

def floatcaster(features):
    casted = [tf.cast(features[column], tf.float32)  for column in features if tf.debugging.is_numeric_tensor(features[column])]
    #print('casted',casted)
    features = tf.concat(  tf.transpose(casted), 1)    
    return features
  
def parse_batched(features,covlist,snplist,cpglist,labellist):
    feature_dict = {}
    feature_dict['cov_input'] = floatcaster( {key:features[key] for key in covlist} )
    feature_dict['snp_input'] = floatcaster( {key:features[key] for key in snplist} )
    feature_dict['cpg_input'] = floatcaster( {key:features[key] for key in cpglist} )
    labels = features[labellist[0]]
    return (feature_dict,labels)
    



In [4]:
filenames = ['../data/dataset.tfrecords']
raw_dataset = tf.data.TFRecordDataset(filenames)
full_dataset = raw_dataset.map(lambda x: parse_single(x,featurelist+labellist))
#print('full',full_dataset)
batched_dataset = full_dataset.repeat().shuffle(100).batch(100).map(lambda features:parse_batched(features,covlist,snplist,cpglist,labellist))
#print('batched2',next(iter(batched_dataset2)))
train = batched_dataset.take(3)
val = batched_dataset.skip(3)

print('cov dim',train.element_spec[0]['cov_input'].shape[1])
print('snp dim',train.element_spec[0]['snp_input'].shape[1])
print('cpg dim',train.element_spec[0]['cpg_input'].shape[1])
#print('training sample',next(iter(train)))

cov dim 12
snp dim 6
cpg dim 7


In [5]:
def create_cov_model(dataset):
    input_dim = dataset.element_spec[0]['cov_input'].shape[1]
    print('inputsize',input_dim)    
    inputs = layers.Input(shape=(input_dim,),name='cov_input')
    x = layers.Dense(1,name='cov_output')(inputs)
    return keras.models.Model(inputs,x,name='cov_model')
cov_model =create_cov_model(train)
print(cov_model.summary())

def create_snp_model(dataset):
    input_dim = dataset.element_spec[0]['snp_input'].shape[1]
    print('inputsize',input_dim)    
    inputs = layers.Input(shape=(input_dim,),name='snp_input')
    x = layers.Dense(32,name='snp_gene')(inputs)
    x = layers.Dense(1,name='snp_gene_output')(x)
    return keras.models.Model(inputs,x,name='snp_model')
snp_model = create_snp_model(train)
print(snp_model.summary())

def create_cpg_model(dataset):
    input_dim = dataset.element_spec[0]['cpg_input'].shape[1]
    print('inputsize',input_dim)    
    inputs = layers.Input(shape=(input_dim,),name='cpg_input')
    x = layers.Dense(32,name='cpg_gene')(inputs)
    x = layers.Dense(1,name='cpg_gene_output')(x)
    return keras.models.Model(inputs,x,name='cpg_model')
cpg_model = create_cpg_model(train)
print(cpg_model.summary())

def create_pathway_model(cov_model,snp_model,cpg_model):
    pathwayInput = layers.concatenate([snp_model.output,cpg_model.output],name='pathway_input')
    x = layers.Dense(8,name='pathway_level_1')(pathwayInput)
    x = layers.Dense(4,name='pathway_level_2')(x)
    x = layers.concatenate([cov_model.output,x],name='pathway_output')
    x = layers.Dense(1)(x)
    #return keras.models.Model(inputs=[cov_model.input,snp_model.input,cpg_model.input],outputs=x)
    return keras.models.Model(inputs={'cov_input':cov_model.input,'snp_input':snp_model.input,'cpg_input':cpg_model.input},outputs=x,name='pathway_model')
pathway_model = create_pathway_model(cov_model,snp_model,cpg_model)

print('Pathway model inputs:',pathway_model.inputs)
print('Pathway model architecture',pathway_model.summary())

inputsize 12


Model: "cov_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ cov_input (InputLayer)          │ (None, 12)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cov_output (Dense)              │ (None, 1)              │            13 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13 (52.00 B)

 Trainable params: 13 (52.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ snp_input           │ (None, 6)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cpg_input           │ (None, 7)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ snp_gene (Dense)    │ (None, 32)        │        224 │ snp_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cpg_gene (Dense)    │ (None, 32)        │        256 │ cpg_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ snp_gene_output     │ (None, 1)         │         33 │ snp_gene[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cpg_gene_output     │ (None, 1)         │         33 │ cpg_gene[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pathway_input       │ (None, 2)         │          0 │ snp_gene_output[… │
│ (Concatenate)       │                   │            │ cpg_gene_output[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cov_input           │ (None, 12)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pathway_level_1     │ (None, 8)         │         24 │ pathway_input[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cov_output (Dense)  │ (None, 1)         │         13 │ cov_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pathway_level_2     │ (None, 4)         │         36 │ pathway_level_1[… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pathway_output      │ (None, 5)         │          0 │ cov_output[0][0], │
│ (Concatenate)       │                   │            │ pathway_level_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1)         │          6 │ pathway_output[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 625 (2.44 KB)

 Trainable params: 625 (2.44 KB)

 Non-trainable params: 0 (0.00 B)

Pathway model architecture None


In [7]:
#print(next(iter(cov_train)).numpy())
pathway_model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
pathway_model.fit(train ,validation_data=val,steps_per_epoch=3,validation_steps=2,epochs=5)
#model.fit(train,validation_data=val,steps_per_epoch=4,validation_steps=3,epochs=5)

Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 27s 10s/step - accuracy: 0.1692 - loss: 13.2455 - val_accuracy: 0.8850 - val_loss: 1.8334
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 39s 15s/step - accuracy: 0.1667 - loss: 13.2853 - val_accuracy: 0.8600 - val_loss: 2.2319
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 41s 16s/step - accuracy: 0.1788 - loss: 13.0927 - val_accuracy: 0.8800 - val_loss: 1.9131
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 41s 16s/step - accuracy: 0.1679 - loss: 13.2654 - val_accuracy: 0.8700 - val_loss: 2.0725
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 31s 11s/step - accuracy: 0.1637 - loss: 13.3318 - val_accuracy: 0.9000 - val_loss: 1.5942
